TODO:

1. lower-case (will get worse)

2. n-gram for external data

####  Settings

In [1]:
lan = 'en'
ngram = 5

####  Load training data and change to lower case

In [2]:
# https://www.kaggle.com/tunguz/ru-baseline-lb-0-9632
import pandas as pd
import numpy as np

train = pd.read_csv('../input/' + lan + '_train.csv', encoding='utf-8')

#### Save n-gram dict

In [3]:
grouped = train.groupby('sentence_id')

d = [{} for _ in range(ngram + 1)] # add 1 so d[n] for n-gram

# For each dict
for i in range(2, ngram + 1):
    print('Working on ' + str(i) + '-gram')
    
    # For each sentence
    for name, group in grouped:
        before = group['before'].values.tolist()
        after  = group['after'].values.tolist() 
        
        # For tokens
        for j in range(len(before) - i + 1):
            key = [before[j + k] for k in range(i)]
            value = [after[j + k] for k in range(i)]
            d[i][tuple(key)] = value

# d[2] # check 2-gram dict for debug

Working on 2-gram
Working on 3-gram
Working on 4-gram
Working on 5-gram


####  Load test data and change to lower case

In [4]:
test = pd.read_csv('../input/' + lan + '_test.csv')
test['id'] = test['sentence_id'].astype(str) + '_' + test['token_id'].astype(str)

#### Load bigdatatrick output

In [5]:
bigdata = pd.read_csv('baseline_ext_en.csv')
test['after'] = bigdata['after'].values.tolist()

#### Cover bigdatatrick output by n-gram results

In [6]:
before = test['before'].values.tolist()
token_id = test['token_id'].values.tolist()
after = test['after'].values.tolist()

for i in range(2, ngram + 1):
    print('Working on ' + str(i) + '-gram')
    
    for j in range(0, len(test) - i + 1): 
        # n-grams to check
        key = [before[j + k] for k in range(i)]
        key = tuple(key)
        
        # Need to check token_id is 0 (tokens should be in on sentence)
        if key in d[i] and (not 0 in token_id[j: j+i]):  
            for k in range(i):
                after[j + k] = d[i][key][k]

    test['after'] = after

Working on 2-gram
Working on 3-gram
Working on 4-gram
Working on 5-gram


#### Save to output

In [7]:
test[['id','after']].to_csv('submission_' + str(ngram) + 'gram-bigdata.csv', index=False)

In [8]:
print('done')

done
